### Import Packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Read dataset

In [3]:
df=pd.read_csv("airline_passengers.csv")
df.head()

### EDA

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df["Month"]=pd.to_datetime(df["Month"])
df.set_index(['Month'],inplace=True)
df.info()

In [ ]:
df.head()

In [ ]:
df.index.freq = 'MS'

In [ ]:
df.isnull().sum()

### Plot the data

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df['Thousands of Passengers'])
plt.title("Monthly total of Airline Passengers")
plt.ylabel("In Thousands")
plt.xlabel("year")
plt.show()

In [4]:
df.index[df.index.month==12]

### Seasonality check

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(df['Thousands of Passengers'])
plt.title("Monthly total of Airline Passengers")
plt.ylabel("In Thousands")
plt.xlabel("year")
for x in df.index[df.index.month==12]:
    plt.axvline(x=x, color='red');
plt.show();

### Decompose the signal

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(df['Thousands of Passengers'], model='additive')
fig, axs = plt.subplots(2, 2,figsize=(15,8))
axs[0, 0].plot(result.observed)
axs[0, 0].autoscale(axis='x',tight=True)
axs[0, 0].set_title('Observed')
axs[0, 1].plot(result.trend,'tab:orange')
axs[0, 1].autoscale(axis='x',tight=True)
axs[0, 1].set_title('Trend')
axs[1, 0].plot(result.seasonal, 'tab:green')
axs[1, 0].autoscale(axis='x',tight=True)
axs[1, 0].set_title('Seasonal')
axs[1, 1].plot(result.resid, 'tab:red')
axs[1, 1].autoscale(axis='x',tight=True)
axs[1, 1].set_title('Residuals')
plt.show()

### Shift the data

In [5]:
df["Thousands of Passengers"].shift()

In [ ]:
df["Thousands of Passengers"].shift().shift()

### Check for Stationarity

In [ ]:
from statsmodels.tsa.stattools import adfuller
def adf_test(df):
    result=adfuller(df)
    print("P Value: ",result[1])
    if result[1]<=0.05:
        print("Strong evidence aganist Null Hypothesis. So, reject Null Hypothesis and conclude data is stationary.")
        return(True)
    else:
        print("Weak evidence aganist Null Hypothesis. So, accept Null Hypothesis and conclude data is non-stationary.")
        return(False)
adf_test(df)

### Automate conversion from non-stationary data to stationary data

In [ ]:
def convert_non_stationary_to_stationary(df):
    d=0
    new_df=df
    while True:
        new_df=new_df-new_df.shift()
        new_df.dropna(inplace=True)
        d=d+1
        if adf_test(new_df):
            print("d-value is",d)
            break

In [ ]:
convert_non_stationary_to_stationary(df)

### Split the data into train and test datasets

In [ ]:
train = df.iloc[:len(df)-30]
test = df.iloc[len(df)-30:]

### Auto ARIMA Model

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from pmdarima import auto_arima
auto_arima(df['Thousands of Passengers'],seasonal=True,m=12).summary()

### Fit the model with train dataset

In [ ]:
model = SARIMAX(train['Thousands of Passengers'],order=(2, 1, 1),seasonal_order=(0, 1, [], 12))
results = model.fit()
results.summary()

### Predict the model with test dataset

In [ ]:
start=len(train)
end=len(train)+len(test)-1
predicted_values = results.predict(start=start, end=end)

In [6]:
ax = test['Thousands of Passengers'].plot(figsize=(12,5))
predicted_values.plot()
plt.legend()
ax.autoscale(axis='x',tight=True)

### Evaluate the model

In [ ]:
import sklearn as sk
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
print("mean_squared_error :",mean_squared_error(test['Thousands of Passengers'],predicted_values ))
print("root_mean_squared_error :",mean_squared_error(test['Thousands of Passengers'],predicted_values, squared=False))
print("mean_absolute_error :",mean_absolute_error(test['Thousands of Passengers'],predicted_values))
print("mean_absolute_percentage_error :",mean_absolute_percentage_error(test['Thousands of Passengers'],predicted_values))

### Retrain the model with entire dataset

In [ ]:
model = SARIMAX(df['Thousands of Passengers'],order=(2, 1, 1),seasonal_order=(0, 1, [], 12))
results = model.fit()
results.summary()

# Forecast the Future...😎

In [ ]:
predicted_values = results.predict(start=len(df), end=len(df)+30)
df['Thousands of Passengers'].plot(figsize=(12,6))
predicted_values.plot()
plt.legend()
plt.show()